In [90]:
import pandas as pd
import glob
import os 
import re

In [156]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', 60)

In [105]:
cwd = os.getcwd()
events = []
os.chdir(cwd)
events = []
#convert all txt files in cwd to df, append to events, then concatenate
for file in glob.glob("*.txt"):
    data = pd.read_csv(file, encoding ='latin1')
    events.append(data)
events = pd.concat(events)
events = events.reset_index()

In [158]:
events.sample(3)

,index,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ELEMID,CHARTTIME,REALTIME,CGID,CORRECTION,CUID,CATEGORY,TITLE,TEXT,EXAM_NAME,PATIENT_INFO
2496,186,8370,4174.0,10407.0,0.0,3108-06-12 16:07:00 EST,3108-06-12 16:52:00 EST,2412.0,N,69.0,Nursing/Other,NaN,\r\n\r\nMICUB 0700-1900 RN Note\r\n\r\nNeuro: off sedat...,NaN,NaN
5309,35,8904,24556.0,11070.0,0.0,3367-01-27 05:48:00 EST,3367-01-27 06:00:00 EST,6326.0,N,69.0,Nursing/Other,NaN,"\r\n\r\n1900-0700\r\n\r\nNeuro: Pt awake, alert, followi...",NaN,NaN
4719,30,8790,27956.0,10930.0,0.0,2707-09-15 15:50:00 EST,2707-09-15 16:35:00 EST,5918.0,N,69.0,Nursing/Other,NPN 7A-7P,\r\nNPN 7A-7P\r\nSig event: Sedation increased as self e...,NaN,NaN


In [92]:
events.columns

Index(['index', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ELEMID', 'CHARTTIME',
       'REALTIME', 'CGID', 'CORRECTION', 'CUID', 'CATEGORY', 'TITLE', 'TEXT',
       'EXAM_NAME', 'PATIENT_INFO'],
      dtype='object')

In [349]:
discharge = events.loc[events.CATEGORY == 'DISCHARGE_SUMMARY'].TEXT.tolist()
for i in range(len(discharge)):
    discharge[i] = discharge[i].replace('\n','newline').replace('\r',' ')

In [350]:
diagnosis_raw = []
for i in discharge:
    if re.search(r'DISCHARGE DIAGNOSES(.*?)DISCHARGE CONDITION', i, re.IGNORECASE):
        diagnosis_raw.append(re.search(r'DISCHARGE DIAGNOSES(.*?)DISCHARGE CONDITION', i, re.IGNORECASE).group(1))
    if re.search(r'DISCHARGE DIAGNOSiS(.*?)DISCHARGE CONDITION', i, re.IGNORECASE):
        diagnosis_raw.append(re.search(r'DISCHARGE DIAGNOSiS(.*?)DISCHARGE CONDITION', i, re.IGNORECASE).group(1))

In [371]:
diagnosis = []
for j in diagnosis_raw:
    
    start = []
    end = []
    for m in re.finditer('newline', j):
        start.append(m.start())
        end.append(m.end())
        
    diagnosis_ind = []
    for i in range(len(start)-1):
        diagnosis_ind.append((j[end[i]:start[i+1]]))
        
    diagnosis.append(diagnosis_ind)

In [321]:
#some issue I can't figure out, should be a simpler way to do the previous cell
#but skips every other new line :(
re.findall(r'newline(.*?)newline', diagnosis_raw[50])

['1. Delirium secondary to hyponatremia probable secondary to  ',
 '2. L1 vertebral compression fracture  ',
 '4. hypertension ',
 '6. urinary incontinence ',
 '8.  peripheral neuropathy ',
 '10. Mitral valve regurgitation ',
 '  ']

In [373]:
#needs further cleaning
diagnosis

[['left foot gangrene ',
  'urosepsis ',
  'a-fibrillation ',
  'ESRD/HD ',
  'diabetes type 2 with triopathy ',
  'MRSA ',
  'depression ',
  'glaucoma ',
  ' ',
  '  '],
 ['MRSA bacteremia ',
  'Pseudomonal lung infection ',
  '1. Small cell lung cancer status post Cisplatin in [**2515**] and  ',
  'BT16 that was complicated by peripheral neuropathy.  ',
  '2. Chronic obstructive pulmonary disease.  ',
  '3. Paroxysmal atrial fibrillation.  ',
  '4. Hypertension.  ',
  '5. Hypothyroidism.  ',
  '6. Anemia.  ',
  '7. Non-melanoma skin cancer.  ',
  '8. Neuropathy  ',
  ' ',
  '  '],
 ['Death ', '  '],
 ['Ventricular fibrillation leading to severe anoxic brain injury. ', '  '],
 ['Thoracic Aortic Aneurysm - s/p Endovascular Stent, Postop  ',
  'Pneumonia(Pseudomonas), Chronic Obstructive Pulmonary Disease,  ',
  'Emphysema, History of Asbestosis versus Mesotheilioma - s/p RUL  ',
  'lung resection, Hypertension,  Renal Cell Carcinoma - s/p  ',
  'Nephrectomy, Depression ',
  '  '],
 ['